In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.qvalue_head = nn.Linear(features_dim, num_outputs)
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.qvalue_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        qvalues = self.qvalue_head(features)
        return qvalues, state

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C17']))
    env = PovOnlyWrapper(env)
    env = IgluActionWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.dqn import DQNTrainer

In [7]:
from ray.tune.integration.wandb import WandbLogger

tune.run(DQNTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "exploration_config": {
                    # The Exploration class to use.
                    "type": "EpsilonGreedy",
                    "initial_epsilon": 1.0,
                    "final_epsilon": 0.02,
                    "epsilon_timesteps": 200000,  # Timesteps over which to anneal epsilon.
             },
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "DQN C17 pretrained (AnnaCNN)"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
DQN_my_env_79012_00000,PENDING,


2021-09-18 21:43:34,141	INFO wandb.py:170 -- Already logged into W&B.
2021-09-18 21:43:34,159	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: WARNING Tried to auto resume run with id e5f53_00000 but id 79012_00000 is set.
(pid=192079) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=192079)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=192079) 2021-09-18 21:43:37,678	INFO dqn.py:188 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=192079) 2021-09-18 21:43:37,678	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=192076) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=192076)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=192079) 2021-09-18 21:43:45,536	INFO trainable.py:109 -- Trainable.setup took 10.375 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=192079) 2021-09-18 21:43:45,537	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=192076) /root/miniconda/envs/py37/lib/python3.7/site-packages/g

Result for DQN_my_env_79012_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-09-18_21-44-56
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: 98a41dadbf62479daa2164e8e1e6c341
  hostname: linar-Z390-GAMING-X
  info:
    last_target_update_ts: 1000
    learner:
      default_policy:
        learner_stats:
          cur_lr: 0.0005
          max_q: 4.673000812530518
          mean_q: -0.2864062786102295
          min_q: -6.367076873779297
        mean_td_error: 2.974947452545166
        td_error: "[10.027929    2.64336    -0.36019802  7.262098   -4.910972    8.456795\n\
          \ -6.7695312  -1.2066615   7.0700674   7.518149    6.6573467  -1.8900771\n\
          \ -0.9292927  -1.1667905   2.3895497  -0.4885858   0.4589427   1.8361897\n\
          \  7.0835295   4.2463355   8.84908     3.988898    8.994044   -0.9691594

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_my_env_79012_00000,RUNNING,192.168.3.5:192079,351,5214.02,351000,-0.01,0,-1,995.74


Result for DQN_my_env_79012_00000:
  agent_timesteps_total: 352000
  custom_metrics: {}
  date: 2021-09-18_23-11-01
  done: false
  episode_len_mean: 995.74
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: -0.01
  episode_reward_min: -1.0
  episodes_this_iter: 1
  episodes_total: 358
  experiment_id: 98a41dadbf62479daa2164e8e1e6c341
  hostname: linar-Z390-GAMING-X
  info:
    last_target_update_ts: 351784
    learner:
      default_policy:
        learner_stats:
          cur_lr: 0.0005
          max_q: 323968812711936.0
          mean_q: 151137013465088.0
          min_q: 46382425571328.0
        mean_td_error: -96602161152.0
        td_error: "[-1.1834313e+12  9.6820475e+11  2.3339540e+12  2.2371117e+12\n  2.2371117e+12\
          \  1.7532568e+12 -1.5168281e+11 -2.1350140e+12\n  2.3339540e+12  9.6820475e+11\
          \  1.7532568e+12 -1.1834313e+12\n  1.9062441e+12 -4.3291844e+12  9.6820475e+11\
          \ -1.0158940e+12\n -4.3291844e+12 -1.1834313e+12 -4.32918

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_my_env_79012_00000,RUNNING,192.168.3.5:192079,352,5227.09,352000,-0.01,0,-1,995.74


2021-09-18 23:11:08,744	WARNING tune.py:519 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2021-09-18 23:11:08,744	WARNING tune.py:519 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2021-09-18 23:11:09,049	WARNING tune.py:519 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 
Process _WandbLoggingProcess-1:
Traceback (most recent call last):
  File "/root/miniconda/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/tune/integration/wandb.py", line 200, in

KeyboardInterrupt: 